# Combinación de conjuntos de datos: Concat y Append

Algunos de los estudios de datos más interesantes provienen de la combinación de diferentes fuentes de datos. Estas operaciones pueden implicar cualquier cosa, desde una concatenación muy sencilla de dos conjuntos de datos diferentes, hasta uniones y fusiones de estilo de base de datos más complicadas que manejen correctamente cualquier superposición entre los conjuntos de datos. ``Series`` y ``DataFrame``. Los correos electrónicos se construyen con este tipo de operación en mente, y Pandas incluye funciones y métodos que hacen que este tipo de disputa de datos sea rápido y sencillo.

Aquí vamos a echar un vistazo a la concatenación simple de ``Series`` y ``DataFramees`` con la función ``pd.concat``; más adelante nos sumergiremos en fusiones y uniones en memoria más sofisticadas implementadas en Pandas.

Comenzamos con las importaciones estándar:

In [1]:
import pandas as pd
import numpy as np

Por conveniencia, definiremos esta función que crea un ``DataFrame`` de una forma particular que será de utilidad a continuación:

In [2]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# example DataFrame
make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


Además, crearemos una clase rápida que nos permita mostrar múltiples ``DataFrame`` uno al lado del otro. El código hace uso del método especial _repr_html_, que utiliza IPython para implementar su visualización de objetos enriquecidos:

In [3]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

El uso de esto se hará más claro a medida que continuamos.

## Recordar: Concatenación de NumPy Arrays 

concatenación de objetos ``Series`` y ``DataFrame`` es muy similar a la concatenación de matrices de Numpy, que se puede hacer a través de ``np.concatenate``. Recuerde que con él, puede combinar el contenido de dos o más matrices en una sola matriz: 

In [4]:
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

El primer argumento es una lista o tupla de matrices para concatenar. Además, se necesita un ``axis`` palabra clave que le permite especificar el eje a lo largo del cual se concatenará el resultado:

In [5]:
x = [[1, 2],
     [3, 4]]
np.concatenate([x, x], axis=1)

array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

## Concatenación simple con ``pd.concat``

Pandas tiene una función, ``pd.concat()``, que tiene una sintaxis similar a ``np.concatenate`` pero contiene una serie de opciones que discutiremos en un momento: 

```python
# Signature in Pandas v0.18
pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
```

``pd.concat()`` se puede utilizar para una concatenación simple de ``Series`` o ``DataFrame``, al igual que ``np.concatenate()`` se puede utilizar para concatenaciones simples de matrices: 

In [6]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

También funciona para concatenar objetos de dimensiones superiores, como ``DataFrames``:

In [9]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
display('df1', 'df2', 'pd.concat([df1, df2])')

,A,B
1,A1,B1
2,A2,B2
,A,B
3,A3,B3
4,A4,B4
,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


De forma predeterminada, la concatenación se realiza por filas dentro del ``DataFrame``(es decir, ``axis=0``). Me gusta ``np.concatenate``, ``pd.concat`` permite la especificación de un eje a lo largo del cual tendrá lugar la concatenación. Considere el siguiente ejemplo:

In [12]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
display('df3', 'df4', "pd.concat([df3, df4], axis=1)")

df3
    A   B
0  A0  B0
1  A1  B1

df4
    C   D
0  C0  D0
1  C1  D1

pd.concat([df3, df4], axis=1)
    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1

### Índices duplicados

Una diferencia importante entre ``np.concatenate`` y ``pd.concat`` es que la concatenación de Pandas conserva los índices , ¡incluso si el resultado tendrá índices duplicados! Considere este ejemplo simple: 

In [13]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index  # make duplicate indices!
display('x', 'y', 'pd.concat([x, y])')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
0,A2,B2
1,A3,B3


Observe los índices repetidos en el resultado. Si bien esto es válido dentro ``DataFrames``, el resultado es a menudo indeseable. ``pd.concat()`` nos da algunas maneras de manejarlo.

#### Capturar las repeticiones como un error

Si desea simplemente verificar que los índices en el resultado de ``pd.concat()`` no se superponen, puede especificar la bandera ``verify_integrity``. Con este conjunto en True, la concatenación generará una excepción si hay índices duplicados. Aquí hay un ejemplo, donde para mayor claridad capturaremos e imprimiremos el mensaje de error: 

In [14]:
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: Int64Index([0, 1], dtype='int64')


#### Ignorando el índice

A veces, el índice en sí mismo no importa y preferiría que simplemente se ignorara. Esta opción se puede especificar mediante la bandera``ignore_index``. Con este valor verdadero, la concatenación creará un nuevo índice entero para el resultado ``Series``:

In [15]:
display('x', 'y', 'pd.concat([x, y], ignore_index=True)')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


#### Agregar claves MultiIndex

Otra opción es utilizar la opción ``keys`` para especificar una etiqueta para las fuentes de datos; el resultado será una serie indexada jerárquicamente que contiene los datos:

In [16]:
display('x', 'y', "pd.concat([x, y], keys=['x', 'y'])")

x
    A   B
0  A0  B0
1  A1  B1

y
    A   B
0  A2  B2
1  A3  B3

pd.concat([x, y], keys=['x', 'y'])
      A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3

El resultado es un indexado múltiple ``DataFrame``, y podemos usar las herramientas discutidas en Indexación jerárquica para transformar estos datos en la representación que nos interesa.

### Concatenación con joins

En los ejemplos simples que acabamos de ver, principalmente estábamos concatenando ``DataFrames`` con nombres de columna compartidos. En la práctica, los datos de diferentes fuentes pueden tener diferentes conjuntos de nombres de columna y ``pd.concat`` ofrece varias opciones en este caso. Considere la concatenación de los dos siguientes ``DataFrames``, que tienen algunas (¡pero no todas!) columnas en común:

In [8]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
display('df5', 'df6', 'pd.concat([df5, df6])')

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6])
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4

De forma predeterminada, las entradas para las que no hay datos disponibles se rellenan con valores NA. Para cambiar esto, podemos especificar una de varias opciones para el ``join`` y ``join_axes`` parámetros de la función concatenar. Por defecto, la unión es una unión de las columnas de entrada ( ``join='outer'``), pero podemos cambiar esto a una intersección de las columnas usando ``join='inner'``:

In [17]:
display('df5', 'df6',
        "pd.concat([df5, df6], join='inner')")

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6], join='inner')
    B   C
1  B1  C1
2  B2  C2
3  B3  C3
4  B4  C4

Otra opción es especificar directamente el índice de las columnas restantes usando el argumento ``join_axes``, que toma una lista de objetos de índice. Aquí especificaremos que las columnas devueltas deben ser las mismas que las de la primera entrada:

In [19]:
display('df5', 'df6',
        "pd.concat([df5, df6], join_axes=[df5.columns])")

TypeError: concat() got an unexpected keyword argument 'join_axes'

La combinación de opciones de la función ``pd.concat`` permite una amplia gama de posibles comportamientos al unir dos conjuntos de datos; téngalos en cuenta cuando utilice estas herramientas para sus propios datos.

### El metodo ``append()``

Debido a que la concatenación directa de arreglos es tan común en ``Series`` y ``DataFrame`` los objetos tienen un método ``append`` que puede lograr lo mismo con menos pulsaciones de teclas. Por ejemplo, en lugar de llamar ``pd.concat([df1, df2])``, simplemente puede llamar ``df1.append(df2)``: 

In [20]:
display('df1', 'df2', 'df1.append(df2)')

<string>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<string>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


,A,B
1,A1,B1
2,A2,B2
,A,B
3,A3,B3
4,A4,B4
,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


Tenga en cuenta que a diferencia de los métodos ``append()`` y ``extend()`` de las listas de Python. El método ``append()`` en Pandas no modifica el objeto original, sino que crea un nuevo objeto con los datos combinados. Tampoco es un método muy eficiente, porque implica la creación de un nuevo índice y un búfer de datos. Por lo tanto, si planea hacer varias operaciones, generalmente es mejor construir una lista de DataFrames y pasarlos todos a la vez con la función ``concat()``.